Turns out we had more data to use than I previously thought. This is just a copy of the previous notebook, after merging all data into a folder called 'data/all_data/'

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os, os.path as osp

In [7]:
path_ims = 'data/all_data/'

See here: https://sites.google.com/view/kpis2024/data?authuser=0, each subfolder contains a wsi split into patches. It is important to be careful and not put those patches in different train/val sets.

In [8]:
path_ims_56nx = osp.join(path_ims, '56Nx')
path_ims_dn = osp.join(path_ims, 'DN')
path_ims_nep25 = osp.join(path_ims, 'NEP25')
path_ims_normal = osp.join(path_ims, 'normal')

Come up with a list of paths to all slides

In [9]:
slides_list_56_nx = os.listdir(path_ims_56nx)
slides_list_56_nx = [osp.join(path_ims_56nx, n) for n in os.listdir(path_ims_56nx)]

slides_list_dn = os.listdir(path_ims_dn)
slides_list_dn = [osp.join(path_ims_dn, n) for n in os.listdir(path_ims_dn)]

slides_list_nep25 = os.listdir(path_ims_nep25)
slides_list_nep25 = [osp.join(path_ims_nep25, n) for n in os.listdir(path_ims_nep25)]

slides_list_normal = os.listdir(path_ims_normal)
slides_list_normal = [osp.join(path_ims_normal, n) for n in os.listdir(path_ims_normal)]

all_slides_list = slides_list_56_nx+slides_list_dn+slides_list_nep25+slides_list_normal
len(all_slides_list) # was 30 before

38

Random split these into five data folds:

In [20]:
num_slides = len(all_slides_list)
meh, val1  = train_test_split(all_slides_list, test_size=num_slides//5, random_state=10)
meh, val2  = train_test_split(meh,    test_size=num_slides//5, random_state=10)
meh, val3  = train_test_split(meh,    test_size=num_slides//5, random_state=10)
val5, val4 = train_test_split(meh,test_size=num_slides//5, random_state=10)

Build validation and train sets, not as slides but as patches

In [21]:
train1 = val2+val3+val4+val5
train2 = val3+val4+val5+val1
train3 = val4+val5+val1+val2
train4 = val5+val1+val2+val3
train5 = val1+val2+val3+val4

In [22]:
def get_img_mask_list_from_slide(slide_path):
    im_path = osp.join(slide_path, 'img')
    mask_path = osp.join(slide_path, 'mask')
    
    im_list_this_slide = sorted(os.listdir(im_path))
    mask_list_this_slide = sorted(os.listdir(mask_path))
    
    im_list_this_slide = [osp.join(slide_path, 'img', n) for n in im_list_this_slide]
    mask_list_this_slide = [osp.join(slide_path, 'mask', n) for n in mask_list_this_slide]
    return im_list_this_slide, mask_list_this_slide

In [23]:
# generate dataframes
columns = ('im_path', 'mask_path')

In [24]:
tr_patches_f1, tr_masks_f1 = [], []
for slide in train1:
    patches, masks = get_img_mask_list_from_slide(slide)
    tr_patches_f1.extend(patches)
    tr_masks_f1.extend(masks)

df_tr_f1 = pd.DataFrame(data=zip(tr_patches_f1, tr_masks_f1), columns=columns)
df_tr_f1.shape

(5509, 2)

In [25]:
def build_patchwise_df(slide_list):
    patch_list, mask_list = [], []
    for slide in slide_list:
        patches, masks = get_img_mask_list_from_slide(slide)
        patch_list.extend(patches)
        mask_list.extend(masks)

    return pd.DataFrame(data=zip(patch_list, mask_list), columns=columns)

In [26]:
df_tr_f1 = build_patchwise_df(train1)
df_tr_f1.shape

(5509, 2)

In [28]:
## NOW
df_tr_f1 = build_patchwise_df(train1)
df_tr_f2 = build_patchwise_df(train2)
df_tr_f3 = build_patchwise_df(train3)
df_tr_f4 = build_patchwise_df(train4)
df_tr_f5 = build_patchwise_df(train5)
df_tr_f1.shape, df_tr_f2.shape, df_tr_f3.shape, df_tr_f4.shape, df_tr_f5.shape

((5509, 2), (5982, 2), (5443, 2), (5372, 2), (5590, 2))

That looks quite well-balanced. Let us build validation dataframes.

In [30]:
df_vl_f1 = build_patchwise_df(val1)
df_vl_f2 = build_patchwise_df(val2)
df_vl_f3 = build_patchwise_df(val3)
df_vl_f4 = build_patchwise_df(val4)
df_vl_f5 = build_patchwise_df(val5)
df_vl_f1.shape, df_vl_f2.shape, df_vl_f3.shape, df_vl_f4.shape, df_vl_f5.shape

((1465, 2), (992, 2), (1531, 2), (1602, 2), (1384, 2))

Allrighty, ready to go, let us save these.

In [31]:
df_tr_f1.to_csv('data/tr_all_f1.csv', index=False)
df_tr_f2.to_csv('data/tr_all_f2.csv', index=False)
df_tr_f3.to_csv('data/tr_all_f3.csv', index=False)
df_tr_f4.to_csv('data/tr_all_f4.csv', index=False)
df_tr_f5.to_csv('data/tr_all_f5.csv', index=False)

In [32]:
df_vl_f1.to_csv('data/vl_all_f1.csv', index=False)
df_vl_f2.to_csv('data/vl_all_f2.csv', index=False)
df_vl_f3.to_csv('data/vl_all_f3.csv', index=False)
df_vl_f4.to_csv('data/vl_all_f4.csv', index=False)
df_vl_f5.to_csv('data/vl_all_f5.csv', index=False)